In [3]:
from __future__ import division
import re, os, sys, string, gensim
import pandas as pd
import numpy as np
from itertools import chain
from collections import Counter
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
import matplotlib.pyplot as plt
# %matplotlib inline

# preprocessing library for tweet data
# https://pypi.org/project/tweet-preprocessor/
import preprocessor as p
from gensim import corpora
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer
lemma = WordNetLemmatizer()
from nltk import word_tokenize, ngrams

pd.set_option('display.max_columns', 20)
pd.set_option('display.max_rows', 20)

Using TensorFlow backend.


## Reading File which contains Tweet for Amazon Prime Sales Day 

In [4]:
input_file_path = '../data/output_#primeday_.csv'
data_tweet = pd.read_csv(input_file_path,error_bad_lines=False,sep=';')

Skipping line 6485: expected 10 fields, saw 11



In [5]:
data_tweet.head(3)

,username,date,retweets,favorites,text,geo,mentions,hashtags,id,permalink
0,frlorente,2018-07-18 11:37,0,0,Imagen del éxito del #PrimeDay de @AmazonESP :...,NaN,@AmazonESP @AmazonEnLucha,#PrimeDay #HuelgaAmazon,1019463538052657152,https://twitter.com/frlorente/status/101946353...
1,Beyonsike,2018-07-18 11:36,0,0,uhh idk how this twitter shit work #LOVEis #Al...,NaN,NaN,#LOVEis #AllStarGame #ASG2018 #LevelUp #TrumpA...,1019463505555197952,https://twitter.com/Beyonsike/status/101946350...
2,startuprad_io,2018-07-18 11:36,1,1,Employees of #Amazon in Leipzig go in #strike ...,NaN,NaN,#Amazon #strike #PrimeDay,1019463404266905600,https://twitter.com/startuprad_io/status/10194...


In [6]:
# In above dataframe, "text" column contains tweet 
data_tweet.drop_duplicates('text',inplace=True)

In [7]:
print("No of Tweets are : {}".format(data_tweet.shape[0]))

No of Tweets are : 6320


## Preprocessing tweets for finding sentiment - Cleaning of Tweets

In [8]:
stop = set(stopwords.words('english'))
add_stop=['http','u','s']
stop = stop.union(add_stop)
# exclude = set(string.punctuation)
lemma = WordNetLemmatizer()

def cleaning_tweets(tweet):
    # Setting Preprocessor paramters - parts to clean from tweet 
    p.set_options(p.OPT.URL, p.OPT.NUMBER)
    sentence = str(tweet)
    sentence = p.clean(sentence)
    reg_http=r"(http)(s)?(://)?"
    sentence = re.sub(reg_http,"", sentence)
    stop_free = " ".join([i for i in sentence.lower().split() if i not in stop])
    #sentence = re.sub("[^A-Za-z .#]+","", stop_free)
    return stop_free

In [9]:
data_tweet['tweet_preprocessed'] = data_tweet['text'].apply(cleaning_tweets)

In [10]:
data_tweet[['text','tweet_preprocessed']].head(10)

,text,tweet_preprocessed
0,Imagen del éxito del #PrimeDay de @AmazonESP :...,imagen del éxito del #primeday de @amazonesp :...
1,uhh idk how this twitter shit work #LOVEis #Al...,uhh idk twitter shit work #loveis #allstargame...
2,Employees of #Amazon in Leipzig go in #strike ...,employees #amazon leipzig go #strike #primeday...
3,Conjunto de utensilios de cocina baja un 17% h...,conjunto de utensilios de cocina baja un % 21 ...
4,So far 100% of my packages from #primeday are ...,far % packages #primeday shipped $ amzn logist...
5,Na Black Friday en Singles Day hebben we nu bl...,na black friday en singles day hebben nu blijk...
6,As Bezos Becomes Richest Man in Modern History...,"bezos becomes richest man modern history, amaz..."
7,#PrimeDay is bad for 1 thing though: taking ad...,#primeday bad thing though: taking advantage a...
8,Almost bought a ukulele on $ amzn bcuz #PrimeDay,almost bought ukulele $ amzn bcuz #primeday
9,"Castle 240RGB ou Castle 280RGB, quelle taille ...","castle rgb ou castle rgb, quelle taille est ce..."


## Sentiment Anlaysis using Vader

In [11]:
analyzer = SentimentIntensityAnalyzer()
def find_polarity_score(tweet):
    score_cleaned = analyzer.polarity_scores(str(tweet))
    return score_cleaned['pos'],score_cleaned['neg'],score_cleaned['neu'],score_cleaned['compound']

In [12]:
data_tweet['positive_score'],data_tweet['negative_score'],data_tweet['neutral_score'],data_tweet['compound_score']=\
zip(*data_tweet['tweet_preprocessed'].apply(find_polarity_score))

In [13]:
data_tweet.head(3)

,username,date,retweets,favorites,text,geo,mentions,hashtags,id,permalink,tweet_preprocessed,positive_score,negative_score,neutral_score,compound_score
0,frlorente,2018-07-18 11:37,0,0,Imagen del éxito del #PrimeDay de @AmazonESP :...,NaN,@AmazonESP @AmazonEnLucha,#PrimeDay #HuelgaAmazon,1019463538052657152,https://twitter.com/frlorente/status/101946353...,imagen del éxito del #primeday de @amazonesp :...,0.000,0.000,1.000,0.0000
1,Beyonsike,2018-07-18 11:36,0,0,uhh idk how this twitter shit work #LOVEis #Al...,NaN,NaN,#LOVEis #AllStarGame #ASG2018 #LevelUp #TrumpA...,1019463505555197952,https://twitter.com/Beyonsike/status/101946350...,uhh idk twitter shit work #loveis #allstargame...,0.000,0.263,0.737,-0.6124
2,startuprad_io,2018-07-18 11:36,1,1,Employees of #Amazon in Leipzig go in #strike ...,NaN,NaN,#Amazon #strike #PrimeDay,1019463404266905600,https://twitter.com/startuprad_io/status/10194...,employees #amazon leipzig go #strike #primeday...,0.146,0.151,0.704,0.2263


## Making class of Sentiment

In [14]:
neg_beg = -1
neg_end = -0.4
neu_end = 0.4
pos_end = 1
scale=[neg_beg,neg_end,neu_end,pos_end]
labels=['negative','neutral','positive']
data_tweet['sentiment']= pd.cut(data_tweet['compound_score'], bins = scale,labels=labels,include_lowest=True)

In [15]:
data_tweet['sentiment'].value_counts()

neutral     3503
positive    2407
negative     410
Name: sentiment, dtype: int64

## Making different files for each sentiment

In [16]:
for label in labels:
    print(label)
    classified_df = data_tweet[data_tweet['sentiment'] == label]
    classified_df.to_csv('../output/'+label+'_tweets.csv')

negative
neutral
positive


## Percentage distribution of sentiment

In [17]:
no_pos = data_tweet[data_tweet['sentiment']=='positive'].shape[0]
no_neg = data_tweet[data_tweet['sentiment']=='negative'].shape[0]
no_neu = data_tweet[data_tweet['sentiment']=='neutral'].shape[0]
no_tweets = data_tweet.shape[0]
pos_per = (no_pos/no_tweets)*100
neg_per = (no_neg/no_tweets)*100
neu_per = (no_neu/no_tweets)*100

In [18]:
print("Total No of tweets is {}".format(no_tweets))
print("Percentage of Positive Tweets {}%".format(pos_per))
print("Percentage of Negative Tweets {}%".format(neg_per))
print("Percentage of Neutral Tweets {}%".format(neu_per))

Total No of tweets is 6320
Percentage of Positive Tweets 38.085443038%
Percentage of Negative Tweets 6.48734177215%
Percentage of Neutral Tweets 55.4272151899%


In [19]:
height = [pos_per,neu_per,neg_per]
bars = ['Positive','Neutral','Negative']
y_pos = np.arange(len(bars))
bar_width = 0.30
plt.figure(figsize=(8, 7))
plt.bar(y_pos, height,bar_width,color=['blue','orange','red'],alpha=0.5,align='center')

plt.xticks(y_pos, bars)
plt.suptitle('Twitter Sentiment : Amazon Prime Day', fontsize=18,fontweight="bold")
plt.ylabel('Percentage of Tweets (%)',fontsize=12)
# plt.tight_layout()
# plt.show()
plt.savefig('../output/sentiment_stat.png')

## LDA - Topic Modelling

### Preprocessing for Topic Modelling

In [20]:
def rem_manual_words(tweet):
    com_words = ['#amazon','amazon','#primeday','#','.','primeday','@amazon','@primeday','prime','day','%',\
                 '#primeday2018','get','take','ends','end','shop','shopping','\x80\xe2','\u2026',\
                '\xe2\x80\xa6','#amazonprimeday','still','hour','hours','sale','today','make','even',
                'use','today','deal','save','purchase','got','far','buy'] 
    return " ".join(i for i in tweet.split() if i not in com_words)

def cleaning(tweet):
    sentence = re.sub("[^A-Za-z\ ]+","", tweet)
    return " ".join(lemma.lemmatize(word) for word in sentence.split())

data_tweet['tweet_preprocessed_remove_common_words'] = data_tweet['tweet_preprocessed'].apply(rem_manual_words)
data_tweet['tweet_preprocessed_remove_common_words'] = data_tweet['tweet_preprocessed_remove_common_words'].apply(cleaning)

In [21]:
data_tweet['tweet_preprocessed_remove_common_words'].head()

0    imagen del xito del de amazonesp descuentos en...
1    uhh idk twitter shit work loveis allstargame a...
2    employee leipzig go strike demand better work ...
3    conjunto de utensilios de cocina baja un bbsbr...
4    package shipped amzn logistics watch uspsupsfe...
Name: tweet_preprocessed_remove_common_words, dtype: object

In [22]:
doc_complete = list(data_tweet['tweet_preprocessed_remove_common_words'])
doc_clean = [doc.split() for doc in doc_complete]
dictionary = corpora.Dictionary(doc_clean)
doc_term_matrix = [dictionary.doc2bow(doc) for doc in doc_clean]
Lda = gensim.models.ldamodel.LdaModel

In [37]:
# dictionary.token2id['basketball']
# dictionary.doc2bow(doc)
# doc
doc_term_matrix[10]

[(121, 1), (122, 1), (123, 2), (124, 1), (125, 1), (126, 1)]

In [38]:
help(Lda)

Help on class LdaModel in module gensim.models.ldamodel:

class LdaModel(gensim.interfaces.TransformationABC, gensim.models.basemodel.BaseTopicModel)
 |  The constructor estimates Latent Dirichlet Allocation model parameters based
 |  on a training corpus:
 |  
 |  >>> lda = LdaModel(corpus, num_topics=10)
 |  
 |  You can then infer topic distributions on new, unseen documents, with
 |  
 |  >>> doc_lda = lda[doc_bow]
 |  
 |  The model can be updated (trained) with new documents via
 |  
 |  >>> lda.update(other_corpus)
 |  
 |  Model persistency is achieved through its `load`/`save` methods.
 |  
 |  Method resolution order:
 |      LdaModel
 |      gensim.interfaces.TransformationABC
 |      gensim.utils.SaveLoad
 |      gensim.models.basemodel.BaseTopicModel
 |      __builtin__.object
 |  
 |  Methods defined here:
 |  
 |  __getitem__(self, bow, eps=None)
 |      Return topic distribution for the given document `bow`, as a list of
 |      (topic_id, topic_probability) 2-tuples.
 

In [39]:
ldamodel = Lda(doc_term_matrix, num_topics=7, id2word = dictionary, passes = 100,alpha='auto',eval_every = 8)

In [40]:
ldamodel.alpha

array([ 0.10005151,  0.12048697,  0.10523123,  0.12867727,  0.18636753,
        0.13314549,  0.40479572])

In [41]:
ldamodel.print_topics(num_topics=7, num_words=20)

[(0,
  u'0.011*"yordiprimeday" + 0.011*"exafm" + 0.007*"worldemojiday" + 0.006*"sandal" + 0.006*"tuesdaythoughts" + 0.005*"fatkiddeals" + 0.005*"latest" + 0.005*"amazonprimedaysale" + 0.005*"docker" + 0.005*"nintendo" + 0.004*"coupon" + 0.004*"tech" + 0.004*"gift" + 0.004*"switch" + 0.004*"und" + 0.004*"amazonecho" + 0.004*"mit" + 0.003*"nicht" + 0.003*"amazondeals" + 0.003*"etsy"'),
 (1,
  u'0.006*"via" + 0.006*"baby" + 0.005*"plus" + 0.005*"gaming" + 0.004*"amazon" + 0.004*"instant" + 0.004*"primedaydeals" + 0.004*"pot" + 0.004*"pro" + 0.004*"water" + 0.004*"foundation" + 0.004*"fortnite" + 0.003*"le" + 0.003*"exclusive" + 0.003*"complete" + 0.003*"top" + 0.003*"raw" + 0.003*"youshopamazongives" + 0.003*"collection" + 0.003*"prime"'),
 (2,
  u'0.019*"jaybird" + 0.016*"worker" + 0.011*"strike" + 0.010*"bezos" + 0.008*"food" + 0.008*"low" + 0.007*"condition" + 0.007*"man" + 0.007*"pay" + 0.006*"eye" + 0.006*"richest" + 0.006*"modern" + 0.005*"history" + 0.005*"mark" + 0.005*"becomes" +